## BERT MODEL

In [43]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.linear_model import Ridge, Lasso
from sklearn import preprocessing
from sklearn.utils import resample
import pandas as pd
import xgboost
import numpy as np 
import nltk
import gensim 
from gensim.models import Word2Vec
from keras.preprocessing.text import text_to_word_sequence
import string
from nltk import word_tokenize
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import linear_model
import numpy as np
from sklearn.model_selection import train_test_split
import scipy
from sklearn.metrics import log_loss
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Read Dataset

In [39]:
df = pd.read_csv('C:\\Users\\bshukla\\Desktop\\Adv_BA\\train_MaefO4x\\train.csv')
df_joke = pd.read_csv('C:\\Users\\bshukla\\Desktop\\Adv_BA\\train_MaefO4x\\jokes.csv')

In [40]:
nn = pd.merge(df, df_joke, how='left',
        left_on='joke_id', right_on='joke_id')
display(nn)

,id,user_id,joke_id,Rating,joke_text
0,31030_110,31030,110,2.750,"Judy was having trouble with her computer, so ..."
1,16144_109,16144,109,5.094,One day the first grade teacher was reading th...
2,23098_6,23098,6,-6.438,Q. What is orange and sounds like a parrot? A....
3,14273_86,14273,86,4.406,Two attorneys went into a diner and ordered tw...
4,18419_134,18419,134,9.375,A man is driving in the country one evening wh...
...,...,...,...,...,...
1092054,9517_132,9517,132,3.156,"One day, three men went to a shrine to ask the..."
1092055,27767_118,27767,118,-1.594,"An American, a Scot and a Canadian were in a t..."
1092056,10580_81,10580,81,2.000,"A Panda walks into a bar, sits down at a table..."
1092057,31007_119,31007,119,8.906,A group of girlfriends is on vacation when the...


In [46]:
SPECIAL_TOKENS = {
    'quoted': 'quoted_item',
    'non-ascii': 'non_ascii_word',
    'undefined': 'something'
}
def clean(text, stem_words=True):
    import re
    from string import punctuation
    from nltk.stem import SnowballStemmer
    from nltk.corpus import stopwords
    
    def pad_str(s):
        return ' '+s+' '
    
    if pd.isnull(text):
        return ''

#    stops = set(stopwords.words("english"))
    # Clean the text, with the option to stem words.
    
    # Empty question
    
    if type(text) != str or text=='':
        return ''

    # Clean the text
    text = re.sub("\'s", " ", text) # we have cases like "Sam is" or "Sam's" (i.e. his) these two cases aren't separable, I choose to compromise are kill "'s" directly
    text = re.sub(" whats ", " what is ", text, flags=re.IGNORECASE)
    text = re.sub("\'ve", " have ", text)
    text = re.sub("can't", "can not", text)
    text = re.sub("n't", " not ", text)
    text = re.sub("i'm", "i am", text, flags=re.IGNORECASE)
    text = re.sub("\'re", " are ", text)
    text = re.sub("\'d", " would ", text)
    text = re.sub("\'ll", " will ", text)
    text = re.sub("e\.g\.", " eg ", text, flags=re.IGNORECASE)
    text = re.sub("b\.g\.", " bg ", text, flags=re.IGNORECASE)
    text = re.sub("(\d+)(kK)", " \g<1>000 ", text)
    text = re.sub("e-mail", " email ", text, flags=re.IGNORECASE)
    text = re.sub("(the[\s]+|The[\s]+)?U\.S\.A\.", " America ", text, flags=re.IGNORECASE)
    text = re.sub("(the[\s]+|The[\s]+)?United State(s)?", " America ", text, flags=re.IGNORECASE)
    text = re.sub("\(s\)", " ", text, flags=re.IGNORECASE)
    text = re.sub("[c-fC-F]\:\/", " disk ", text)
    
    # remove comma between numbers, i.e. 15,000 -> 15000
    
    text = re.sub('(?<=[0-9])\,(?=[0-9])', "", text)
    
#     # all numbers should separate from words, this is too aggressive
    
#     def pad_number(pattern):
#         matched_string = pattern.group(0)
#         return pad_str(matched_string)
#     text = re.sub('[0-9]+', pad_number, text)
    
    # add padding to punctuations and special chars, we still need them later
    
    text = re.sub('\$', " dollar ", text)
    text = re.sub('\%', " percent ", text)
    text = re.sub('\&', " and ", text)
    
#    def pad_pattern(pattern):
#        matched_string = pattern.group(0)
#       return pad_str(matched_string)
#    text = re.sub('[\!\?\@\^\+\*\/\,\~\|\`\=\:\;\.\#\\\]', pad_pattern, text) 
        
    text = re.sub('[^\x00-\x7F]+', pad_str(SPECIAL_TOKENS['non-ascii']), text) # replace non-ascii word with special word
    
    # indian dollar
    
    text = re.sub("(?<=[0-9])rs ", " rs ", text, flags=re.IGNORECASE)
    text = re.sub(" rs(?=[0-9])", " rs ", text, flags=re.IGNORECASE)
    
    # clean text rules get from : https://www.kaggle.com/currie32/the-importance-of-cleaning-text
    text = re.sub(r" (the[\s]+|The[\s]+)?US(A)? ", " America ", text)
    text = re.sub(r" UK ", " England ", text, flags=re.IGNORECASE)
    text = re.sub(r" india ", " India ", text)
    text = re.sub(r" switzerland ", " Switzerland ", text)
    text = re.sub(r" china ", " China ", text)
    text = re.sub(r" chinese ", " Chinese ", text) 
    text = re.sub(r" imrovement ", " improvement ", text, flags=re.IGNORECASE)
    text = re.sub(r" intially ", " initially ", text, flags=re.IGNORECASE)
    text = re.sub(r" quora ", " Quora ", text, flags=re.IGNORECASE)
    text = re.sub(r" dms ", " direct messages ", text, flags=re.IGNORECASE)  
    text = re.sub(r" demonitization ", " demonetization ", text, flags=re.IGNORECASE) 
    text = re.sub(r" actived ", " active ", text, flags=re.IGNORECASE)
    text = re.sub(r" kms ", " kilometers ", text, flags=re.IGNORECASE)
    text = re.sub(r" cs ", " computer science ", text, flags=re.IGNORECASE) 
    text = re.sub(r" upvote", " up vote", text, flags=re.IGNORECASE)
    text = re.sub(r" iPhone ", " phone ", text, flags=re.IGNORECASE)
    text = re.sub(r" \0rs ", " rs ", text, flags=re.IGNORECASE)
    text = re.sub(r" calender ", " calendar ", text, flags=re.IGNORECASE)
    text = re.sub(r" ios ", " operating system ", text, flags=re.IGNORECASE)
    text = re.sub(r" gps ", " GPS ", text, flags=re.IGNORECASE)
    text = re.sub(r" gst ", " GST ", text, flags=re.IGNORECASE)
    text = re.sub(r" programing ", " programming ", text, flags=re.IGNORECASE)
    text = re.sub(r" bestfriend ", " best friend ", text, flags=re.IGNORECASE)
    text = re.sub(r" dna ", " DNA ", text, flags=re.IGNORECASE)
    text = re.sub(r" III ", " 3 ", text)
    text = re.sub(r" banglore ", " Banglore ", text, flags=re.IGNORECASE)
    text = re.sub(r" J K ", " JK ", text, flags=re.IGNORECASE)
    text = re.sub(r" J\.K\. ", " JK ", text, flags=re.IGNORECASE)
    
    # replace the float numbers with a random number, it will be parsed as number afterward, and also been replaced with word "number"
    
    text = re.sub('[0-9]+\.[0-9]+', " 87 ", text)
  
    
    # Remove punctuation from text
    text = ''.join([c for c in text if c not in punctuation]).lower()
       # Return a list of words
    return text


In [47]:
nn["Rating"]

0          2.750
1          5.094
2         -6.438
3          4.406
4          9.375
           ...  
1092054    3.156
1092055   -1.594
1092056    2.000
1092057    8.906
1092058   -3.344
Name: Rating, Length: 1092059, dtype: float64

In [48]:
nn['category'] = pd.cut(nn.Rating, bins=[-10,0,10], labels = ['bad','good'])
nn

,id,user_id,joke_id,Rating,joke_text,category
0,31030_110,31030,110,2.750,"Judy was having trouble with her computer, so ...",good
1,16144_109,16144,109,5.094,One day the first grade teacher was reading th...,good
2,23098_6,23098,6,-6.438,Q. What is orange and sounds like a parrot? A....,bad
3,14273_86,14273,86,4.406,Two attorneys went into a diner and ordered tw...,good
4,18419_134,18419,134,9.375,A man is driving in the country one evening wh...,good
...,...,...,...,...,...,...
1092054,9517_132,9517,132,3.156,"One day, three men went to a shrine to ask the...",good
1092055,27767_118,27767,118,-1.594,"An American, a Scot and a Canadian were in a t...",bad
1092056,10580_81,10580,81,2.000,"A Panda walks into a bar, sits down at a table...",good
1092057,31007_119,31007,119,8.906,A group of girlfriends is on vacation when the...,good


In [49]:
nna = nn.copy()
nna

,id,user_id,joke_id,Rating,joke_text,category
0,31030_110,31030,110,2.750,"Judy was having trouble with her computer, so ...",good
1,16144_109,16144,109,5.094,One day the first grade teacher was reading th...,good
2,23098_6,23098,6,-6.438,Q. What is orange and sounds like a parrot? A....,bad
3,14273_86,14273,86,4.406,Two attorneys went into a diner and ordered tw...,good
4,18419_134,18419,134,9.375,A man is driving in the country one evening wh...,good
...,...,...,...,...,...,...
1092054,9517_132,9517,132,3.156,"One day, three men went to a shrine to ask the...",good
1092055,27767_118,27767,118,-1.594,"An American, a Scot and a Canadian were in a t...",bad
1092056,10580_81,10580,81,2.000,"A Panda walks into a bar, sits down at a table...",good
1092057,31007_119,31007,119,8.906,A group of girlfriends is on vacation when the...,good


In [50]:
nn = nn.astype(str)
nn
nn = nn.sample(n = 5000, random_state=1) 
nn
nn['category'].value_counts()


good    3382
bad     1613
nan        5
Name: category, dtype: int64

In [51]:
nn = nn[nn.category != "nan"]
nn['category'].value_counts()

good    3382
bad     1613
Name: category, dtype: int64

In [52]:
from simpletransformers.classification import MultiLabelClassificationModel

df = nn[["joke_text","category"]]
df['category'].value_counts()

good    3382
bad     1613
Name: category, dtype: int64

In [53]:
from sklearn.model_selection import train_test_split
df
le = preprocessing.LabelEncoder()
df['label'] = le.fit_transform(df.category.values)
df

C:\Users\bshukla\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,joke_text,category,label
167920,Q: If a person who speaks three languages is c...,bad,0
649069,When my three-year-old son opened the birthday...,good,1
110645,A teacher is explaining to her class how diffe...,bad,0
759124,A couple of hunters are out in the woods in th...,good,1
802566,A man approached a very beautiful woman in a l...,good,1
...,...,...,...
936797,A horse walks into a bar. The bartender asks: ...,good,1
809949,A man piloting a hot air balloon discovers he ...,good,1
403982,A man and Cindy Crawford get stranded on a des...,bad,0
703608,A group of girlfriends is on vacation when the...,good,1


In [55]:
df = df[["joke_text","label"]]
df['label'].value_counts()

df = df.astype(str)
df

,joke_text,label
167920,Q: If a person who speaks three languages is c...,0
649069,When my three-year-old son opened the birthday...,1
110645,A teacher is explaining to her class how diffe...,0
759124,A couple of hunters are out in the woods in th...,1
802566,A man approached a very beautiful woman in a l...,1
...,...,...
936797,A horse walks into a bar. The bartender asks: ...,1
809949,A man piloting a hot air balloon discovers he ...,1
403982,A man and Cindy Crawford get stranded on a des...,0
703608,A group of girlfriends is on vacation when the...,1


In [56]:
nnn_majority = df[df.label=='1']
nnn_minority = df[df.label=='0']


nnn_minority
df_majority_downsampled = resample(nnn_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=1613,     # to match minority class
                                 random_state=123) # reproducible results
df1 = pd.concat([df_majority_downsampled, nnn_minority])

df1.label.value_counts()

0    1613
1    1613
Name: label, dtype: int64

In [57]:
df1['joke_text'] = df['joke_text'].apply(clean)

In [58]:
df1

,joke_text,label
884522,a neutron walks into a bar and orders a drink ...,1
986834,a drunk staggers into a catholic church enters...,1
320174,a little boy goes to his dad and asks what is ...,1
713933,a man is driving in the country one evening wh...,1
320377,a man and cindy crawford get stranded on a des...,1
...,...,...
145237,q did you hear about the dyslexic devil worshi...,0
559822,just a thought before criticizing someone walk...,0
729266,q if a person who speaks three languages is ca...,0
335972,it was the day of the big sale rumors of the s...,0


SPLIT INTO TRAIN & VALIDATION

In [59]:


train, test = train_test_split(df1, test_size=0.4, stratify = df1['label'])


train['joke_text'].apply(str)
test['joke_text'].apply(str)

train["label"] = pd.to_numeric(train["label"])
test["label"] = pd.to_numeric(test["label"])

test

C:\Users\bshukla\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\bshukla\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,joke_text,label
942553,just a thought before criticizing someone walk...,1
719702,a man piloting a hot air balloon discovers he ...,1
136863,as a premed student i had to take a difficult ...,1
481721,how many irishmen does it take to change a lig...,0
177441,a little boy goes to his dad and asks what is ...,1
...,...,...
30721,q if a person who speaks three languages is ca...,0
206366,how many men does it take to screw in a light ...,0
303539,a couple of hunters are out in the woods in th...,1
377941,a drunk staggers into a catholic church enters...,0


BERT MODEL IMPLEMENTATION

In [19]:
from simpletransformers.classification import ClassificationModel
model = ClassificationModel('roberta', 'roberta-base', use_cuda = False, 
                            args = {'overwrite_output_dir': True, 'num_train_epochs': 2})
model.train_model(train)
result, model_outputs, wrong_predictions = model.eval_model(test)

Features loaded from cache at cache_dir/cached_train_roberta_128_2_2997


Running loss: 0.719585


Running loss: 0.811537

Training of roberta model complete. Saved to outputs/.
Converting to features started.


In [21]:
result

{'mcc': 0.0930216852616408,
 'tp': 1287,
 'tn': 63,
 'fp': 582,
 'fn': 66,
 'eval_loss': 0.6111494753360748}

In [36]:

modeltwo = ClassificationModel('roberta', 'roberta-base', use_cuda = False, args = {'overwrite_output_dir': True, 'num_train_epochs': 2})
modeltwo.train_model(train)
resulttwo, modeltwo_outputs, wrong_predictions = modeltwo.eval_model(test)

Converting to features started.


Running loss: 0.694951


Running loss: 0.581192

Training of roberta model complete. Saved to outputs/.
Converting to features started.


In [38]:
resulttwo

{'mcc': 0.17583952539119366,
 'tp': 378,
 'tn': 381,
 'fp': 264,
 'fn': 268,
 'eval_loss': 0.6711408387363693}

In [ ]:

model3 = ClassificationModel('roberta', 'roberta-base', use_cuda = False, 
                            args = {'overwrite_output_dir': True, 'num_train_epochs': 2})
model3.train_model(train)
result3, model3_outputs, wrong_predictions = model3.eval_model(test)

Features loaded from cache at cache_dir/cached_train_roberta_128_2_1935


Running loss: 0.655734


Running loss: 0.635190

Training of roberta model complete. Saved to outputs/.
Features loaded from cache at cache_dir/cached_dev_roberta_128_2_1291


BERT MODEL PERFORMANCE

In [ ]:
# result two: 
## {'mcc': 0.5673987968268284,
 #'tp': 871,
 #'tn': 587,
 #'fp': 359,
 #'fn': 75,
 #'eval_loss': 0.8151762967512703}
precision = 871/(871+359)
recall = 871/(871+75)
f1_score = (2*precision*recall)/(precision+recall)
## Result 2 performance 
print("Resulttwo Performance: ", 
      "Precision:", precision, 
      "Recall:", recall, 
      "F1 Score:", f1_score)